# search for specific words

In [ ]:
import tweepy
#import pandas as pd
from collections import defaultdict
import re
import json
import requests
import os
from dotenv import load_dotenv


from pprint import pprint
from datetime import datetime
import urllib.parse

#https://www.thepythoncode.com/article/translate-text-in-python
from googletrans import Translator, constants
from pprint import pprint



In [ ]:
load_dotenv()

API_Key = os.environ.get("API_KEY")
API_Key_Secret = os.environ.get("API_KEY_SECRET")
Bearer_Token = os.environ.get("BEARER_TOKEN")


auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
bearer = 'bearer ' + Bearer_Token


In [ ]:
# init the Google API translator
translator = Translator()

runDate = datetime.today().strftime('%Y-%m-%d')


In [ ]:
def tweetQueryBuilder(searchTerms):
    base_api_url = 'https://api.twitter.com/2/tweets/search/recent?query='
    fixed_args = '-is:retweet  has:links '
    
    query_arg_prefix = fixed_args + searchTerms
    query_arg_prefix = urllib.parse.quote(query_arg_prefix)
    query_arg_suffix = '&max_results=100&tweet.fields=attachments,author_id,context_annotations,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,source,text,withheld&expansions=referenced_tweets.id,geo.place_id'
    
    query = base_api_url + query_arg_prefix + query_arg_suffix
    
    return query


In [ ]:
queries = [
    #blockchain
    'humanitarian response blockchain',
    'humanitarian blockchain',
    'blockchain privacy risk',
    'blockchain refugee',

    #drones
    'humanitarian response drones',
    'humanitarian drone',
    'drone damage assessment',
    'humanitarian drone mapping',

    #mapping
    'humanitarian geospatial',
    'humanitarian mapping',
    'humanitarian crowd map',

    #data
    'humanitarian data collection',
    'humanitarian analyze data',
    'pre-disaster data',
    
        'humanitarian assessment',
    'humanitarian surge',
    'surge assessment',
    'predicting humanitarian',
    'predicting crisis',
    'predicting surge',

    'disaster assessment',
    'disaster blockchain',
    'disaster response'
    
]

languages = ['en']



In [ ]:
total_result_count=0
for query in queries:
    for language in languages:
        translation = translator.translate(query, src="en", dest=language)
        search = tweetQueryBuilder(translation.text)
        print(search)
        
        folder = 'twitter_data/staged_tweets/'


        response = requests.get(search, headers={"Authorization":bearer})


        parsed = json.loads(response.text)
        rc = parsed['meta']['result_count']
        print("rc is " + str(rc))
        total_result_count += rc

        #print(json.dumps(parsed, indent=4, sort_keys=True))

        fullFilePath = folder + runDate + '_' + query + '_' + language +'.json'
        print(fullFilePath)
    
                            

        
        with open(fullFilePath, 'w') as outfile:
            outfile.write(json.dumps(parsed, indent=4, sort_keys=True))
print("Total Results: ", total_result_count)
    